In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [3]:
def navigate_up(path, levels):
    """Navigate up `levels` directories from the given path."""
    for _ in range(levels):
        path = os.path.dirname(path)
    return path

In [4]:
# Get the current working directory
llamaindex_dir = os.getcwd()
# Get the parent directory
llamaindex_dir = os.path.dirname(llamaindex_dir)

sys.path.append(llamaindex_dir + "/utils")
sys.path.append(navigate_up(llamaindex_dir, 2) + "/law-sec-insights/backend")
# sys.path

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [5]:
from llamaindex_utils import *

In [6]:
from app.chat.engine import get_chat_engine

In [7]:
import logging
import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Load data and build an index + Storing your index + Query your data

In [8]:
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, get_response_synthesizer
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor

# documents = SimpleDirectoryReader(llamaindex_dir + "/data").load_data()
# index = VectorStoreIndex.from_documents(documents)

In [9]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader(llamaindex_dir + "/data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    # saving the embeddings to disk
    # By default, this will save the data to the directory storage, but you can change that by passing a persist_dir parameter.
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

```python
# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)
```

How to get lot of data when you have relevant results but potentially no data if you have nothing relevant
- we customize our retriever to use a different number for top_k
  - For a custom retriever, we use `RetrieverQueryEngine`.
- and add a post-processing step that requires that the retrieved nodes reach a minimum similarity score to be included
  - For the post-processing step, we use `SimilarityPostprocessor`
 

[Response Synthesizer](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/): A Response Synthesizer is what generates a response from an LLM, using a user query and a given set of text chunks. The output of a response synthesizer is a Response object. When used in a query engine, the response synthesizer is used after nodes are retrieved from a retriever, and after any node-postprocessors are ran.

In [10]:
# build index
# index = VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# query
response = query_engine.query("What did the author do growing up?")
print(response)

The author was involved in writing and programming activities during their formative years.


In [11]:
# help(RetrieverQueryEngine)

# `get_chat_engine`

- local implementation

There are a huge variety of retrievers that you can learn about in our [module guide on retrievers](https://docs.llamaindex.ai/en/stable/module_guides/querying/retriever/).

In [12]:
from app.models.db import MessageRoleEnum, MessageStatusEnum, MessageSubProcess, MessageSubProcessStatusEnum
from app.schema import Message, Conversation, Document, DocumentMetadataKeysEnum, SecDocumentTypeEnum
from datetime import datetime
from uuid import UUID


mock_message = Message(
    conversation_id=UUID("01234567-89ab-cdef-0123-456789abcdef"),
    content="Hello, how can I help you?",
    role=MessageRoleEnum.assistant,
    status=MessageStatusEnum.SUCCESS,
    sub_processes=[
        MessageSubProcess(
            message_id=UUID("01234567-89ab-cdef-0123-456789abcdef"),
            source="chunking",
            status=MessageSubProcessStatusEnum.FINISHED,
            metadata_map=None
        ),
        MessageSubProcess(
            message_id=UUID("abcdef01-2345-6789-abcd-ef0123456789"),
            source="node_parsing",
            status=MessageSubProcessStatusEnum.FINISHED,
            metadata_map=None
        )
    ]
)

mock_document = Document(
    id=UUID("123e4567-e89b-12d3-a456-426614174000"),
    created_at=datetime.now(),
    updated_at=datetime.now(),
    url="https://example.com/document.pdf",
    metadata_map={
        DocumentMetadataKeysEnum.SEC_DOCUMENT: {
            "title": "Annual Report 2023",
            "author": "Company XYZ",
            "pages": 25,
            "company_name": "Apple",
            "company_ticker": "AAPL",
            "doc_type": SecDocumentTypeEnum.TEN_K,
            "year": 2023
            
        }
    }
)

doc_id_to_index = {"123e4567-e89b-12d3-a456-426614174000": index}

In [13]:
mock_document.id

UUID('123e4567-e89b-12d3-a456-426614174000')

In [14]:
from app.schema import Conversation as ConversationSchema

conversation = ConversationSchema(messages=[mock_message], documents=[mock_document])

In [15]:
from app.chat.notebook_engine import get_chat_engine

[Llama Debug Handler](https://docs.llamaindex.ai/en/stable/examples/callbacks/LlamaDebugHandler/)

In [19]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)

agent = await get_chat_engine(conversation=conversation, doc_id_to_index=doc_id_to_index, callback_handler=llama_debug)

In [20]:
response = agent.chat("What is the revenue of uber in 2021.")
# TODO:
# FIX FOR THIS MOCK

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: qualitative_question_engine with args: {
  "input": "What is the revenue of Uber in 2021?"
}
Generated 1 sub questions.
[123e4567-e89b-12d3-a456-426614174000] Q: What is the revenue of Uber in 2021?
[123e4567-e89b-12d3-a456-426614174000] A: Empty Response
Got output: Empty Response

STARTING TURN 2
---------------

=== Calling Function ===
Calling function: quantitative_question_engine with args: {
  "input": "What is the revenue of Uber in 2021?"
}
Generated 1 sub questions.
[extract_json_from_sec_document[Apple (AAPL) 10-K (2023)]] Q: What is the revenue of Uber in 2021?
ERROR:app.chat.tools:Error retrieving data from polygon.io for document_id 123e4567-e89b-12d3-a456-426614174000
Traceback (most recent call last):
  File "/Users/pierre.krzisch/dev/perso/projects/law-sec-insights/backend/app/chat/tools.py", line 108, in extract_data_from_sec_document
    period_of_report_date=str(sec_metadata.period_of_report